# 06 - Agent Ethics Patterns: Designing Responsible Multi-Agent Systems

**Learning Objectives:**
- Understand ethical principles for agentic AI systems
- Implement accountability, transparency, and fairness patterns
- Design human-in-the-loop (HITL) decision frameworks
- Apply patterns to real-world scenarios (healthcare, bioinformatics)

**Prerequisites:**
- Completed `05-mcp-security-audit.ipynb`
- Understanding of agentic AI concepts

**Time Required:** ~45 minutes

**Context:** This notebook synthesizes ethical patterns applicable to projects like [spatial-mcp](https://github.com/lynnlangit/spatial-mcp) and provides frameworks for responsible agent design.

---

## 1. The Agentic AI Ethics Challenge

As AI systems move from **assistive** (answering questions) to **agentic** (taking actions), ethical considerations multiply.

### Key Differences: Assistive vs Agentic AI

| Aspect | Assistive AI | Agentic AI |
|--------|--------------|------------|
| **Output** | Information, recommendations | Actions, decisions, changes |
| **Reversibility** | User decides to act | Actions may be irreversible |
| **Accountability** | User responsible | Shared/unclear responsibility |
| **Scope** | Single interaction | Multi-step, multi-system |
| **Autonomy** | None | Partial to full |

### Ethical Principles for Agentic AI

Based on IEEE, NIST AI RMF, and EU AI Act:

1. **Transparency**: Users understand what the agent can and will do
2. **Accountability**: Clear responsibility chain for agent actions
3. **Fairness**: Actions don't discriminate or cause unequal harm
4. **Safety**: Agents cannot cause unacceptable harm
5. **Human Control**: Meaningful human oversight is maintained
6. **Privacy**: Agent respects data protection principles

In [ ]:
from dataclasses import dataclass, field
from enum import Enum
from typing import List, Optional, Dict, Any, Callable
from datetime import datetime
import json

print("✅ Ethics pattern library loaded")

## 2. Pattern 1: Graduated Autonomy

Not all actions require the same level of oversight. Implement **graduated autonomy** based on action risk.

In [ ]:
class AutonomyLevel(Enum):
    """Levels of agent autonomy based on action risk."""
    
    # Level 1: Full autonomy - agent acts without asking
    FULL = "full"
    
    # Level 2: Notify - agent acts and informs user
    NOTIFY = "notify"
    
    # Level 3: Confirm - agent proposes, user approves
    CONFIRM = "confirm"
    
    # Level 4: Supervised - human reviews each step
    SUPERVISED = "supervised"
    
    # Level 5: Disabled - action not permitted
    DISABLED = "disabled"

@dataclass
class ActionPolicy:
    """Policy defining autonomy level for an action type."""
    action_type: str
    autonomy_level: AutonomyLevel
    requires_justification: bool
    max_impact_scope: str  # e.g., "single_file", "patient_record", "workflow"
    cooldown_seconds: int = 0  # Minimum time between actions
    daily_limit: Optional[int] = None

# Example policies for a bioinformatics agent (spatial-mcp style)
BIOINFORMATICS_POLICIES = {
    # Read operations - full autonomy
    "read_file": ActionPolicy(
        action_type="read_file",
        autonomy_level=AutonomyLevel.FULL,
        requires_justification=False,
        max_impact_scope="single_file"
    ),
    "query_database": ActionPolicy(
        action_type="query_database",
        autonomy_level=AutonomyLevel.FULL,
        requires_justification=False,
        max_impact_scope="query_result"
    ),
    "search_pubmed": ActionPolicy(
        action_type="search_pubmed",
        autonomy_level=AutonomyLevel.FULL,
        requires_justification=False,
        max_impact_scope="search_result"
    ),
    
    # Analysis operations - notify
    "run_analysis": ActionPolicy(
        action_type="run_analysis",
        autonomy_level=AutonomyLevel.NOTIFY,
        requires_justification=True,
        max_impact_scope="analysis_output",
        cooldown_seconds=60
    ),
    
    # Write operations - confirm required
    "write_results": ActionPolicy(
        action_type="write_results",
        autonomy_level=AutonomyLevel.CONFIRM,
        requires_justification=True,
        max_impact_scope="output_file"
    ),
    "modify_patient_data": ActionPolicy(
        action_type="modify_patient_data",
        autonomy_level=AutonomyLevel.SUPERVISED,
        requires_justification=True,
        max_impact_scope="patient_record"
    ),
    
    # Workflow operations - supervised
    "submit_workflow": ActionPolicy(
        action_type="submit_workflow",
        autonomy_level=AutonomyLevel.SUPERVISED,
        requires_justification=True,
        max_impact_scope="compute_cluster",
        daily_limit=5
    ),
    
    # Dangerous operations - disabled
    "delete_data": ActionPolicy(
        action_type="delete_data",
        autonomy_level=AutonomyLevel.DISABLED,
        requires_justification=True,
        max_impact_scope="any"
    )
}

def check_action_permitted(action_type: str, policies: dict) -> tuple:
    """Check if an action is permitted and at what autonomy level."""
    policy = policies.get(action_type)
    
    if policy is None:
        # Unknown action - default to supervised
        return AutonomyLevel.SUPERVISED, "Unknown action type - defaulting to supervised"
    
    if policy.autonomy_level == AutonomyLevel.DISABLED:
        return AutonomyLevel.DISABLED, f"Action '{action_type}' is not permitted"
    
    return policy.autonomy_level, f"Action permitted at {policy.autonomy_level.value} level"

# Test the policy checker
print("🔐 Graduated Autonomy Policy Checks:")
print("=" * 50)
for action in ["read_file", "write_results", "submit_workflow", "delete_data", "unknown_action"]:
    level, message = check_action_permitted(action, BIOINFORMATICS_POLICIES)
    print(f"  {action}: {level.value} - {message}")

## 3. Pattern 2: Explainable Decision Chain

Every agent decision should be traceable and explainable. Implement a **decision chain** that captures reasoning.

In [ ]:
@dataclass
class DecisionStep:
    """A single step in an agent's decision chain."""
    step_id: str
    timestamp: str
    action: str
    reasoning: str
    inputs: Dict[str, Any]
    outputs: Dict[str, Any]
    confidence: float  # 0.0 to 1.0
    alternatives_considered: List[str]
    human_override: Optional[str] = None

@dataclass
class DecisionChain:
    """Complete decision chain for an agent task."""
    chain_id: str
    user_request: str
    started_at: str
    steps: List[DecisionStep] = field(default_factory=list)
    final_outcome: Optional[str] = None
    completed_at: Optional[str] = None
    
    def add_step(self, action: str, reasoning: str, inputs: dict, 
                 outputs: dict, confidence: float, alternatives: list) -> DecisionStep:
        """Add a step to the decision chain."""
        step = DecisionStep(
            step_id=f"{self.chain_id}-step-{len(self.steps)+1}",
            timestamp=datetime.utcnow().isoformat() + "Z",
            action=action,
            reasoning=reasoning,
            inputs=inputs,
            outputs=outputs,
            confidence=confidence,
            alternatives_considered=alternatives
        )
        self.steps.append(step)
        return step
    
    def explain(self) -> str:
        """Generate human-readable explanation of the decision chain."""
        explanation = [f"Decision Chain: {self.chain_id}"]
        explanation.append(f"User Request: {self.user_request}")
        explanation.append(f"Started: {self.started_at}")
        explanation.append("\nDecision Steps:")
        
        for i, step in enumerate(self.steps, 1):
            explanation.append(f"\n  Step {i}: {step.action}")
            explanation.append(f"    Reasoning: {step.reasoning}")
            explanation.append(f"    Confidence: {step.confidence:.0%}")
            if step.alternatives_considered:
                explanation.append(f"    Alternatives: {', '.join(step.alternatives_considered)}")
            if step.human_override:
                explanation.append(f"    ⚠️ Human Override: {step.human_override}")
        
        if self.final_outcome:
            explanation.append(f"\nFinal Outcome: {self.final_outcome}")
        
        return "\n".join(explanation)

# Example: Decision chain for a bioinformatics query
chain = DecisionChain(
    chain_id="dc-2026-001",
    user_request="Find genes associated with ovarian cancer and analyze their expression in patient PAT001",
    started_at=datetime.utcnow().isoformat() + "Z"
)

# Step 1: Literature search
chain.add_step(
    action="search_pubmed",
    reasoning="User asked about ovarian cancer genes. PubMed is the best source for established gene associations.",
    inputs={"query": "ovarian cancer associated genes BRCA", "max_results": 20},
    outputs={"articles_found": 20, "key_genes": ["BRCA1", "BRCA2", "TP53", "KRAS"]},
    confidence=0.95,
    alternatives=["query_tcga", "search_clinvar"]
)

# Step 2: Patient data retrieval
chain.add_step(
    action="query_patient_expression",
    reasoning="Need to check expression of identified genes in patient PAT001's data.",
    inputs={"patient_id": "PAT001", "genes": ["BRCA1", "BRCA2", "TP53", "KRAS"]},
    outputs={"expression_data": "retrieved", "samples": 3},
    confidence=0.90,
    alternatives=["aggregate_cohort_data"]
)

# Step 3: Analysis (would normally require confirmation)
step3 = chain.add_step(
    action="run_differential_expression",
    reasoning="Differential expression analysis will identify significant changes in gene expression.",
    inputs={"method": "DESeq2", "comparison": "tumor_vs_normal"},
    outputs={"significant_genes": 47, "top_gene": "BRCA1"},
    confidence=0.85,
    alternatives=["edgeR", "limma"]
)

chain.final_outcome = "Identified 47 differentially expressed genes, with BRCA1 showing highest significance."
chain.completed_at = datetime.utcnow().isoformat() + "Z"

print(chain.explain())

## 4. Pattern 3: Ethical Guardrails Framework

Define explicit ethical constraints that the agent must respect.

In [ ]:
@dataclass
class EthicalConstraint:
    """An ethical constraint the agent must respect."""
    name: str
    description: str
    principle: str  # Which ethical principle this supports
    check_function: Callable[[Dict], bool]
    violation_response: str
    severity: str  # "blocking", "warning", "advisory"

def check_patient_consent(context: dict) -> bool:
    """Check if patient consent exists for the action."""
    patient_id = context.get("patient_id")
    action_type = context.get("action_type")
    
    # In production, this would check a consent database
    # For demo, assume consent exists for research use
    if action_type in ["research_analysis", "aggregate_statistics"]:
        return True
    if action_type in ["share_externally", "publish_individually"]:
        return False  # Would need explicit consent
    return True

def check_data_minimization(context: dict) -> bool:
    """Check if only necessary data is being accessed."""
    requested_fields = set(context.get("requested_fields", []))
    necessary_fields = set(context.get("necessary_fields", []))
    
    # Fail if requesting more than necessary
    excess_fields = requested_fields - necessary_fields
    return len(excess_fields) == 0

def check_bias_risk(context: dict) -> bool:
    """Check if action might introduce or amplify bias."""
    # Check for demographic-based filtering that could introduce bias
    filters = context.get("filters", {})
    sensitive_attributes = ["race", "ethnicity", "gender", "socioeconomic"]
    
    for attr in sensitive_attributes:
        if attr in filters:
            return False  # Potential bias risk
    return True

# Define ethical constraints for a healthcare AI agent
HEALTHCARE_ETHICAL_CONSTRAINTS = [
    EthicalConstraint(
        name="patient_consent",
        description="Patient must have consented to the specific use of their data",
        principle="Privacy & Autonomy",
        check_function=check_patient_consent,
        violation_response="Cannot proceed without patient consent for this action",
        severity="blocking"
    ),
    EthicalConstraint(
        name="data_minimization",
        description="Only access data that is necessary for the task",
        principle="Privacy",
        check_function=check_data_minimization,
        violation_response="Requesting more data than necessary - please justify or reduce scope",
        severity="warning"
    ),
    EthicalConstraint(
        name="bias_prevention",
        description="Actions should not introduce or amplify demographic bias",
        principle="Fairness",
        check_function=check_bias_risk,
        violation_response="This action may introduce demographic bias - review required",
        severity="warning"
    )
]

def evaluate_ethical_constraints(context: dict, constraints: list) -> dict:
    """Evaluate all ethical constraints for a given context."""
    results = {
        "all_passed": True,
        "blocking_violations": [],
        "warnings": [],
        "details": []
    }
    
    for constraint in constraints:
        passed = constraint.check_function(context)
        
        result = {
            "constraint": constraint.name,
            "principle": constraint.principle,
            "passed": passed,
            "severity": constraint.severity
        }
        
        if not passed:
            result["violation_response"] = constraint.violation_response
            
            if constraint.severity == "blocking":
                results["all_passed"] = False
                results["blocking_violations"].append(constraint.name)
            elif constraint.severity == "warning":
                results["warnings"].append(constraint.name)
        
        results["details"].append(result)
    
    return results

# Test ethical constraints
test_context = {
    "patient_id": "PAT001",
    "action_type": "research_analysis",
    "requested_fields": ["gene_expression", "diagnosis", "age"],
    "necessary_fields": ["gene_expression", "diagnosis"],
    "filters": {}
}

results = evaluate_ethical_constraints(test_context, HEALTHCARE_ETHICAL_CONSTRAINTS)

print("🔍 Ethical Constraint Evaluation:")
print("=" * 50)
print(f"All Passed: {results['all_passed']}")
print(f"Blocking Violations: {results['blocking_violations']}")
print(f"Warnings: {results['warnings']}")
print("\nDetails:")
for detail in results["details"]:
    status = "✅" if detail["passed"] else "❌"
    print(f"  {status} {detail['constraint']} ({detail['principle']}): {'Passed' if detail['passed'] else detail.get('violation_response', 'Failed')}")

## 5. Pattern 4: Stakeholder Impact Assessment

Before taking significant actions, assess impact on all stakeholders.

In [ ]:
@dataclass
class Stakeholder:
    """A stakeholder who may be affected by agent actions."""
    name: str
    type: str  # "individual", "group", "organization", "society"
    interests: List[str]
    vulnerability_level: str  # "high", "medium", "low"

@dataclass
class ImpactAssessment:
    """Assessment of action impact on a stakeholder."""
    stakeholder: Stakeholder
    impact_type: str  # "positive", "negative", "neutral"
    impact_magnitude: str  # "high", "medium", "low"
    description: str
    mitigation: Optional[str] = None

# Define stakeholders for a healthcare AI system
HEALTHCARE_STAKEHOLDERS = [
    Stakeholder(
        name="Patient",
        type="individual",
        interests=["privacy", "accurate diagnosis", "appropriate treatment", "autonomy"],
        vulnerability_level="high"
    ),
    Stakeholder(
        name="Healthcare Provider",
        type="individual",
        interests=["clinical accuracy", "efficiency", "liability protection", "patient trust"],
        vulnerability_level="medium"
    ),
    Stakeholder(
        name="Healthcare Organization",
        type="organization",
        interests=["regulatory compliance", "reputation", "efficiency", "cost management"],
        vulnerability_level="medium"
    ),
    Stakeholder(
        name="Research Community",
        type="group",
        interests=["data access", "reproducibility", "scientific advancement"],
        vulnerability_level="low"
    ),
    Stakeholder(
        name="Underrepresented Populations",
        type="group",
        interests=["equitable treatment", "representation in training data", "fair outcomes"],
        vulnerability_level="high"
    )
]

def assess_action_impact(action: str, action_details: dict, 
                         stakeholders: list) -> List[ImpactAssessment]:
    """Assess the impact of an action on all stakeholders."""
    assessments = []
    
    # This would be more sophisticated in production
    # Here we demonstrate the pattern with simple heuristics
    
    for stakeholder in stakeholders:
        if action == "train_model_on_patient_data":
            if stakeholder.name == "Patient":
                assessments.append(ImpactAssessment(
                    stakeholder=stakeholder,
                    impact_type="negative" if "privacy" in stakeholder.interests else "neutral",
                    impact_magnitude="medium",
                    description="Patient data used for training may have privacy implications",
                    mitigation="Use differential privacy, obtain informed consent"
                ))
            elif stakeholder.name == "Underrepresented Populations":
                assessments.append(ImpactAssessment(
                    stakeholder=stakeholder,
                    impact_type="negative",
                    impact_magnitude="high" if action_details.get("diverse_data", False) == False else "low",
                    description="Model trained on non-diverse data may perform poorly for underrepresented groups",
                    mitigation="Ensure diverse representation in training data, test for bias"
                ))
            elif stakeholder.name == "Research Community":
                assessments.append(ImpactAssessment(
                    stakeholder=stakeholder,
                    impact_type="positive",
                    impact_magnitude="medium",
                    description="New model may advance research capabilities",
                    mitigation=None
                ))
    
    return assessments

# Example impact assessment
action_details = {
    "data_sources": ["hospital_a", "hospital_b"],
    "diverse_data": False,
    "sample_size": 10000
}

impacts = assess_action_impact("train_model_on_patient_data", action_details, HEALTHCARE_STAKEHOLDERS)

print("📊 Stakeholder Impact Assessment:")
print("=" * 60)
print(f"Action: train_model_on_patient_data")
print("\nImpacts:")
for impact in impacts:
    icon = "🟢" if impact.impact_type == "positive" else "🔴" if impact.impact_type == "negative" else "⚪"
    print(f"\n  {icon} {impact.stakeholder.name}")
    print(f"     Type: {impact.impact_type}, Magnitude: {impact.impact_magnitude}")
    print(f"     Description: {impact.description}")
    if impact.mitigation:
        print(f"     Mitigation: {impact.mitigation}")

## 6. Pattern 5: Reversibility and Recovery

Design agents to prefer reversible actions and maintain recovery capabilities.

In [ ]:
class Reversibility(Enum):
    """Reversibility classification for agent actions."""
    FULLY_REVERSIBLE = "fully_reversible"      # Can undo completely
    PARTIALLY_REVERSIBLE = "partially_reversible"  # Can partially undo
    IRREVERSIBLE = "irreversible"              # Cannot undo

@dataclass
class ActionCheckpoint:
    """Checkpoint for potential rollback."""
    checkpoint_id: str
    timestamp: str
    action: str
    state_before: Dict[str, Any]
    reversibility: Reversibility
    rollback_procedure: Optional[str] = None

class RecoveryManager:
    """Manages checkpoints and recovery for agent actions."""
    
    def __init__(self, max_checkpoints: int = 100):
        self.checkpoints: List[ActionCheckpoint] = []
        self.max_checkpoints = max_checkpoints
    
    def create_checkpoint(self, action: str, state: dict, 
                          reversibility: Reversibility,
                          rollback_procedure: str = None) -> ActionCheckpoint:
        """Create a checkpoint before taking an action."""
        checkpoint = ActionCheckpoint(
            checkpoint_id=f"ckpt-{len(self.checkpoints)+1}-{datetime.utcnow().strftime('%Y%m%d%H%M%S')}",
            timestamp=datetime.utcnow().isoformat() + "Z",
            action=action,
            state_before=state.copy(),
            reversibility=reversibility,
            rollback_procedure=rollback_procedure
        )
        
        self.checkpoints.append(checkpoint)
        
        # Trim old checkpoints
        if len(self.checkpoints) > self.max_checkpoints:
            self.checkpoints = self.checkpoints[-self.max_checkpoints:]
        
        return checkpoint
    
    def can_rollback(self, checkpoint_id: str) -> bool:
        """Check if rollback is possible for a checkpoint."""
        for ckpt in self.checkpoints:
            if ckpt.checkpoint_id == checkpoint_id:
                return ckpt.reversibility != Reversibility.IRREVERSIBLE
        return False
    
    def get_rollback_plan(self, checkpoint_id: str) -> Optional[str]:
        """Get the rollback procedure for a checkpoint."""
        for ckpt in self.checkpoints:
            if ckpt.checkpoint_id == checkpoint_id:
                if ckpt.reversibility == Reversibility.IRREVERSIBLE:
                    return "⚠️ This action cannot be rolled back."
                return ckpt.rollback_procedure or "Restore state_before values"
        return None

# Example usage
recovery = RecoveryManager()

# Checkpoint before a file write (reversible)
ckpt1 = recovery.create_checkpoint(
    action="write_analysis_results",
    state={"file_exists": False, "path": "/results/analysis_001.csv"},
    reversibility=Reversibility.FULLY_REVERSIBLE,
    rollback_procedure="Delete file at /results/analysis_001.csv"
)

# Checkpoint before a database update (partially reversible)
ckpt2 = recovery.create_checkpoint(
    action="update_patient_record",
    state={"patient_id": "PAT001", "previous_status": "active"},
    reversibility=Reversibility.PARTIALLY_REVERSIBLE,
    rollback_procedure="Restore previous_status to 'active'; audit log entry cannot be removed"
)

# Checkpoint before sending an email (irreversible)
ckpt3 = recovery.create_checkpoint(
    action="send_notification_email",
    state={"recipient": "doctor@hospital.org", "subject": "New results available"},
    reversibility=Reversibility.IRREVERSIBLE,
    rollback_procedure=None
)

print("🔄 Recovery Manager Status:")
print("=" * 50)
for ckpt in recovery.checkpoints:
    can_rollback = recovery.can_rollback(ckpt.checkpoint_id)
    icon = "✅" if can_rollback else "⚠️"
    print(f"\n{icon} {ckpt.checkpoint_id}")
    print(f"   Action: {ckpt.action}")
    print(f"   Reversibility: {ckpt.reversibility.value}")
    print(f"   Rollback: {recovery.get_rollback_plan(ckpt.checkpoint_id)}")

## 7. Pattern 6: Ethics Review Gate

For high-stakes actions, implement a formal ethics review gate.

In [ ]:
@dataclass
class EthicsReviewRequest:
    """Request for ethics review of an agent action."""
    request_id: str
    action: str
    context: Dict[str, Any]
    risk_assessment: str
    stakeholder_impacts: List[str]
    requested_by: str  # agent_id
    requested_at: str
    urgency: str  # "immediate", "standard", "low"
    
@dataclass
class EthicsReviewDecision:
    """Decision from ethics review."""
    request_id: str
    decision: str  # "approved", "rejected", "approved_with_conditions"
    reviewer: str
    reviewed_at: str
    conditions: Optional[List[str]] = None
    rationale: str = ""

class EthicsReviewGate:
    """Gate for ethics review of high-stakes actions."""
    
    # Actions that always require ethics review
    MANDATORY_REVIEW_ACTIONS = [
        "train_model_on_patient_data",
        "share_data_externally",
        "automated_clinical_decision",
        "modify_treatment_recommendation",
        "delete_patient_data"
    ]
    
    def __init__(self):
        self.pending_reviews: List[EthicsReviewRequest] = []
        self.completed_reviews: List[EthicsReviewDecision] = []
    
    def requires_review(self, action: str, context: dict) -> bool:
        """Check if an action requires ethics review."""
        # Mandatory review for certain actions
        if action in self.MANDATORY_REVIEW_ACTIONS:
            return True
        
        # Check for high-risk indicators
        if context.get("affects_vulnerable_population", False):
            return True
        if context.get("involves_minors", False):
            return True
        if context.get("potential_for_discrimination", False):
            return True
        if context.get("large_scale_impact", False):
            return True
            
        return False
    
    def submit_for_review(self, action: str, context: dict, 
                          agent_id: str, urgency: str = "standard") -> EthicsReviewRequest:
        """Submit an action for ethics review."""
        request = EthicsReviewRequest(
            request_id=f"er-{len(self.pending_reviews)+1}-{datetime.utcnow().strftime('%Y%m%d')}",
            action=action,
            context=context,
            risk_assessment=self._assess_risk(action, context),
            stakeholder_impacts=self._identify_stakeholder_impacts(action, context),
            requested_by=agent_id,
            requested_at=datetime.utcnow().isoformat() + "Z",
            urgency=urgency
        )
        self.pending_reviews.append(request)
        return request
    
    def _assess_risk(self, action: str, context: dict) -> str:
        """Assess risk level of the action."""
        if action in self.MANDATORY_REVIEW_ACTIONS:
            return "HIGH - Mandatory review action"
        if context.get("affects_vulnerable_population"):
            return "HIGH - Affects vulnerable population"
        return "MEDIUM - Standard review"
    
    def _identify_stakeholder_impacts(self, action: str, context: dict) -> List[str]:
        """Identify potential stakeholder impacts."""
        impacts = []
        if "patient" in action.lower() or context.get("involves_patients"):
            impacts.append("Patients: Privacy and autonomy concerns")
        if context.get("affects_vulnerable_population"):
            impacts.append("Vulnerable populations: Potential for disproportionate impact")
        if "share" in action.lower() or "external" in action.lower():
            impacts.append("Organization: Compliance and liability considerations")
        return impacts

# Example usage
ethics_gate = EthicsReviewGate()

# Test if actions require review
test_actions = [
    ("search_pubmed", {}),
    ("train_model_on_patient_data", {"involves_patients": True}),
    ("run_analysis", {"affects_vulnerable_population": True}),
    ("share_data_externally", {"recipient": "research_partner"})
]

print("🔍 Ethics Review Gate Assessment:")
print("=" * 50)
for action, context in test_actions:
    requires = ethics_gate.requires_review(action, context)
    icon = "⚠️ REVIEW REQUIRED" if requires else "✅ No review needed"
    print(f"\n{action}: {icon}")
    
    if requires:
        request = ethics_gate.submit_for_review(action, context, "agent-001")
        print(f"  Request ID: {request.request_id}")
        print(f"  Risk: {request.risk_assessment}")
        for impact in request.stakeholder_impacts:
            print(f"  Impact: {impact}")

## 8. Applying Patterns to spatial-mcp

Here's how these patterns apply to the [spatial-mcp](https://github.com/lynnlangit/spatial-mcp) project:

In [ ]:
SPATIAL_MCP_ETHICS_CONFIG = """
┌─────────────────────────────────────────────────────────────────────────────┐
│              SPATIAL-MCP ETHICS PATTERN APPLICATION                         │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  MCP SERVER: fgbio                                                          │
│  ├── Pattern 1 (Graduated Autonomy)                                         │
│  │   • read_bam: FULL autonomy                                              │
│  │   • run_analysis: NOTIFY                                                 │
│  │   • write_output: CONFIRM                                                │
│  ├── Pattern 5 (Reversibility)                                              │
│  │   • Create checkpoints before file writes                                │
│  │   • Maintain rollback capability for output files                        │
│                                                                             │
│  MCP SERVER: mockepic (Patient Data)                                        │
│  ├── Pattern 3 (Ethical Constraints)                                        │
│  │   • Data minimization enforced                                           │
│  │   • PHI access logged                                                    │
│  ├── Pattern 4 (Stakeholder Impact)                                         │
│  │   • Patient privacy assessment for each query                            │
│  │   • Vulnerable population checks                                         │
│  ├── Pattern 6 (Ethics Review)                                              │
│  │   • Required for: data sharing, bulk exports                             │
│                                                                             │
│  MCP SERVER: seqera (Workflow Execution)                                    │
│  ├── Pattern 1 (Graduated Autonomy)                                         │
│  │   • list_workflows: FULL                                                 │
│  │   • submit_workflow: SUPERVISED                                          │
│  │   • cancel_workflow: CONFIRM                                             │
│  ├── Pattern 2 (Decision Chain)                                             │
│  │   • Full trace of workflow submission rationale                          │
│  │   • Cost and resource impact documented                                  │
│  ├── Pattern 5 (Reversibility)                                              │
│  │   • Workflow cancellation supported                                      │
│  │   • Output cleanup on failure                                            │
│                                                                             │
│  MCP SERVER: pubmed                                                         │
│  ├── Pattern 1 (Graduated Autonomy)                                         │
│  │   • All operations: FULL (read-only, public data)                        │
│  ├── Pattern 2 (Decision Chain)                                             │
│  │   • Log search queries for reproducibility                               │
│                                                                             │
│  CROSS-CUTTING CONCERNS                                                     │
│  ├── All servers implement audit logging                                    │
│  ├── Decision chains stored for compliance review                           │
│  ├── Ethics review gate for patient data operations                         │
│  └── Stakeholder impact assessment for new analysis types                   │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
"""

print(SPATIAL_MCP_ETHICS_CONFIG)

---

## 🎯 Summary: Ethics Pattern Checklist

Use this checklist when designing any agentic AI system:

In [ ]:
ETHICS_DESIGN_CHECKLIST = """
┌─────────────────────────────────────────────────────────────────────────────┐
│                    AGENTIC AI ETHICS DESIGN CHECKLIST                       │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  ☐ GRADUATED AUTONOMY                                                       │
│    • Classified all actions by risk level                                   │
│    • Defined autonomy levels (full → disabled)                              │
│    • Implemented HITL gates for high-risk actions                           │
│                                                                             │
│  ☐ EXPLAINABLE DECISIONS                                                    │
│    • Decision chains captured for all significant actions                   │
│    • Reasoning documented at each step                                      │
│    • Alternatives considered are recorded                                   │
│    • Human overrides tracked                                                │
│                                                                             │
│  ☐ ETHICAL CONSTRAINTS                                                      │
│    • Consent requirements defined                                           │
│    • Data minimization enforced                                             │
│    • Bias prevention checks implemented                                     │
│    • Blocking vs warning severity defined                                   │
│                                                                             │
│  ☐ STAKEHOLDER IMPACT                                                       │
│    • All stakeholders identified                                            │
│    • Vulnerable populations explicitly considered                           │
│    • Impact assessment for significant actions                              │
│    • Mitigation strategies documented                                       │
│                                                                             │
│  ☐ REVERSIBILITY & RECOVERY                                                 │
│    • Actions classified by reversibility                                    │
│    • Checkpoints created before state changes                               │
│    • Rollback procedures defined                                            │
│    • Irreversible actions require extra confirmation                        │
│                                                                             │
│  ☐ ETHICS REVIEW GATE                                                       │
│    • Mandatory review actions defined                                       │
│    • Review request process implemented                                     │
│    • Escalation path for urgent decisions                                   │
│    • Review decisions logged                                                │
│                                                                             │
│  ☐ COMPLIANCE & AUDIT                                                       │
│    • All actions logged with context                                        │
│    • Audit trail supports compliance requirements                           │
│    • Regular ethics audits scheduled                                        │
│    • Incident response plan documented                                      │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
"""

print(ETHICS_DESIGN_CHECKLIST)

---

## 📚 Key Takeaways

| Pattern | Purpose | When to Use |
|---------|---------|-------------|
| Graduated Autonomy | Match oversight to risk | All agentic systems |
| Decision Chain | Enable accountability | Significant decisions |
| Ethical Constraints | Enforce principles | Data/privacy-sensitive actions |
| Stakeholder Impact | Prevent unintended harm | High-impact actions |
| Reversibility | Enable recovery | State-changing actions |
| Ethics Review Gate | Formal oversight | High-stakes decisions |

## 🔗 Resources

- [spatial-mcp Project](https://github.com/lynnlangit/spatial-mcp) - Reference implementation
- [IEEE Ethically Aligned Design](https://ethicsinaction.ieee.org/)
- [NIST AI Risk Management Framework](https://www.nist.gov/itl/ai-risk-management-framework)
- [EU AI Act](https://artificialintelligenceact.eu/)
- [Anthropic Responsible Scaling Policy](https://www.anthropic.com/news/anthropics-responsible-scaling-policy)

## 🎯 Next Steps

You've now completed the full examples series! Consider:

1. **Integrate patterns into your own projects** - Apply these patterns to your MCP implementations
2. **Contribute to the repo** - Add domain-specific examples or improve existing ones
3. **Stay current** - Follow the evolving regulatory landscape (EU AI Act, NIST updates)